In [3]:
from langchain_openai import AzureChatOpenAI
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain import hub
from dotenv import load_dotenv
import os

load_dotenv()

True

In [4]:
llm = AzureChatOpenAI(
    azure_deployment=os.environ.get("azure_openai_deployment", ""), model="gpt-4o-mini"
)
db = SQLDatabase.from_uri("sqlite:///data/library_database.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [5]:
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")
system_message = prompt_template.format(dialect="SQLite", top_k=5)


In [7]:
#system_message += f"\nFormat instructions : {books_parser.get_format_instructions()}\n"

In [84]:
system_message = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables. If a book is actively rented, you should consider it unavailable.
If the user asks to rent a book you should ask for customer id, if the id is not in the database you should ask for the customer's name and add the customer to the database.
"""

In [8]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(
    llm,
    toolkit.get_tools(),
    state_modifier=system_message
)


In [9]:
import pprint
def invoke(x):
    inv = lambda x: agent_executor.invoke({"messages": [("user", x)]})
    response = inv(x)
    res = response["messages"][-1].content
    pprint.pprint(res)

In [10]:
user_query = "Is one hundred years of solitude available in the library?"
invoke(user_query)


('Yes, "One Hundred Years of Solitude" is available in the library. However, '
 'it is currently rented out.')


In [11]:
user_query = "List all the history books in the library?"
invoke(user_query)

('Here are some history books available in the library:\n'
 '\n'
 '1. **The Histories** by Herodotus\n'
 '2. **The Guns of August** by Barbara W. Tuchman\n'
 '3. **Sapiens: A Brief History of Humankind** by Yuval Noah Harari\n'
 '4. **The Diary of a Young Girl** by Anne Frank\n'
 "5. **A People's History of the United States** by Howard Zinn\n"
 '\n'
 'If you need more details or specific information, feel free to ask!')
